- 지금까지 내용에서 불편한 점은?

1. 단순히 iris,wine 데이터를 받아서 사용했을뿐인데, 직접 공부하면서 코드를 하나씩 실행에 보면 혼돈이 크다는것을 알 수 있다.
2. 쥬피터 노트북 상황에서 데이터의 전처리와 여러 알고라즘의 반복 실행 , 하이퍼 파라미터의 튜닝과정을 번갈아 하다 보면 코드의 실행 순서에 혼돈 발생
3. 이런 경우 class로 만들어 진행에도 되지만,
4. sklearn 유저에게는 꼭 그럴 필요 없이 준비된 기능이 있다 -> Pipeline

In [1]:
# 데이터 읽기

import pandas as pd

red_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/winequality-red.csv'
white_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/winequality-white.csv'

red_wine = pd.read_csv(red_url, sep = ';')
white_wine = pd.read_csv(white_url, sep = ';')

# 와인 색상 구분 컬럼
red_wine['color'] = 1
white_wine['color'] = 0

# 두 데이터 합치기
wine = pd.concat([red_wine, white_wine])

X = wine.drop(['color'], axis=1)
y = wine['color'] # 타겟 데이터

In [2]:
# Scaler(StandardScaler()) -> clf(DecisionTreeClassifier())
# 위와 같은 순서의 파이프라인을 짜고 싶다.

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

estimators = [
    ('scaler', StandardScaler()), # 첫번째 단계의 (객체 이름, 객체)
    ('clf', DecisionTreeClassifier()) # 두번째 단계의 (객체 이름, 객체)
]

pipe = Pipeline(estimators)

그렇다면 pipe라는 변수에는 무엇이 담겨있게 될까?

In [3]:
pipe 

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', DecisionTreeClassifier())])

In [4]:
pipe.steps

[('scaler', StandardScaler()), ('clf', DecisionTreeClassifier())]

In [5]:
pipe.steps[0]

('scaler', StandardScaler())

In [6]:
pipe['scaler']

StandardScaler()

In [7]:
# 아직 분류기만 만들어낸거지 그 안의 파라미터들도 지정해주어야한다.
# 파라미터 지정은 아래와 같은 방법으로 할 수 있다.
# pipe.set_params(객체명__파라미터명 = 값)

pipe.set_params(clf__max_depth = 2)
pipe.set_params(clf__random_state = 13)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', DecisionTreeClassifier(max_depth=2, random_state=13))])

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                   test_size = 0.2, 
                                                   random_state=13,
                                                   stratify= y
                                                   )

In [9]:
# 이전에는 스케일러를 통과시키고 분류기에 학습시켜두었는데
# 그 과정이 Pipe 변수에 저장이 되어있으므로 간단하게 아래 코드로 실행 가능하다.

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', DecisionTreeClassifier(max_depth=2, random_state=13))])

In [10]:
from sklearn.metrics import accuracy_score

y_pred_tr = pipe.predict(X_train)
y_pred_test = pipe.predict(X_test)

print('Train acc : ', accuracy_score(y_train, y_pred_tr))
print('Test acc : ', accuracy_score(y_test, y_pred_test))

Train acc :  0.9657494708485664
Test acc :  0.9576923076923077
